In [ ]:
from datetime import datetime, date
from collections import defaultdict
import numpy as np
import os, string

testset = False

def diff_days(s1,s2):
	date_format = "%Y-%m-%d"
	a = datetime.strptime(s1, date_format)
	b = datetime.strptime(s2, date_format)
	delta = b - a
	return delta.days  # 傳回兩日期天數差
#-----------------------------------------------------
offers_file = "./offers.csv"
transactions_file = "./t1/part_1.csv"     # transactions.csv 
if testset:
	history_file = "./testHistory.csv"
	folder = "./test/"
else:
	history_file = "./trainHistory.csv"
	folder = "./train/"
out_file = os.path.join(folder, "user_features.csv")

with open("./trainHistory.csv","r")as th:
    th.readline()
    for i, lines in enumerate(th):
        li = lines.strip().split(",")
        li[-1].replace('-','/')
#------------------------------------------------------
def generate_features(history_file, transactions_file, out_file):
    
	#keep a dictionary with the offerdata
	offers = {}
	for e, line in enumerate( open(offers_file) ):  # offers.csv
		row = line.strip().split(",")   
		offers[row[0]] = row   # offers[優惠券編號]

	#keep two dictionaries with the shopper id's from train
	ids = {}
	for e, line in enumerate( open(history_file) ):
		if e > 0:
			row = line.strip().split(",")
			ids[row[0]] = row   # ids[id]
	
	#get category-dept map
	cat_dept = {}
	dept_cat = {}
	for e, line in enumerate( open("./data/cat_dept_map.csv","r") ):
		row = line.strip().split(",")
		cat_dept[row[0]] = row[1]  # cat_dept[category] = [dept]
		if not row[1] in dept_cat: # 若商品類別不在dept_cat
			dept_cat[row[1]] = []  
		dept_cat[row[1]].append(row[0]) # dept_cat[dept] = {category}

	seen_ids = set([])
	out_train = open(out_file, "wb")  
	feature_list = ["total_spend_30","dep_spend_30","dep_count_30","visits_30","prodid_spend_30","prodid_count_30","prodid_spend_all","prodid_count_all","prodid_spend_corr"]
	out_train.write(("id," + ",".join(feature_list)+"\n").encode())

	# get product marketshares for correcting prodid_spend
	product_marketshares = {}
	fi = open( os.path.join(folder, "product_features.csv"),"r")
	fi.readline()
	for lines in fi:
		li = lines.strip().split(",")
		product_marketshares[li[0]] = float(li[1])

	#iterate through dataset
	last_id = 0
	features = defaultdict(float)
	days = [0]*30
	filt = open(transactions_file, "rb")
	filt.readline()
	for e, line in enumerate( filt ):
		#poor man's csv reader
		row = line.decode().strip().split(",")
		#write away the features when we get to a new shopper id
		if last_id != row[0] and e != 0:
			if last_id in ids:
				features['visits_30'] = np.sum(days)
				
				outline = str(last_id)
				for l in feature_list:
					if l == "prodid_spend_corr" and not l in features:
						prodid = offers[history[2]][1]+" "+offers[history[2]][3]+" "+offers[history[2]][5]
						outline += ",-"+str(100*product_marketshares[prodid])
					elif l in features:
						outline += ","+str(features[l])
					else:
						outline += ",0"
				
				outline += "\n"
				
				out_train.write(outline.encode())
				seen_ids.add(last_id)

			# reset features
			features = defaultdict(float)
			days = [0]*30
			
		#generate features from transaction record
		if row[0] in ids:
			history = ids[row[0]]

			dep = cat_dept[offers[ history[2] ][1]]
			cats = dept_cat[dep]
			
			date_diff_days = diff_days(row[6],history[-1])
			# get total spend last 30 days
			if date_diff_days < 30:
				features['total_spend_30'] += float( row[10] ) # 總金額
			
			# check how many bought this exact item over last 30 days
			if offers[history[2]][3] == row[4] and offers[ history[2] ][1] == row[3] and offers[ history[2] ][5] == row[5]:
				if date_diff_days < 30:
					features['prodid_spend_30'] += float(row[10]) #總金額
					features['prodid_count_30'] += float(row[9])  #購買個數
				features['prodid_spend_all'] += float(row[10])
				features['prodid_spend_corr'] += float(row[10])
				features['prodid_count_all'] += float(row[9])
			
			# check if this category is in the dept
			if row[3] in cats:
				if date_diff_days < 30:
					features['dep_count_30'] += float( row[9] )
					features['dep_spend_30'] += float( row[10] )
			
			# append this day to list of last 30 days
			if date_diff_days < 30:
				days[date_diff_days] = 1

		last_id = row[0]
		if e % 100000 == 0 and e > 0:
			print(e)

	# write out the last entry
	if last_id in ids:
		features['visits_30'] = np.sum(days)
		
		outline = str(last_id)
		for l in feature_list:
			if l == "prodid_spend_corr" and not l in features:
				prodid = offers[history[2]][1]+" "+offers[history[2]][3]+" "+offers[history[2]][5]
				outline += ",-"+str(100*product_marketshares[prodid])
			elif l in features:
				outline += ","+str(features[l])
			else:
				outline += ",0"
		
		outline += "\n"
		
		out_train.write( outline )
		seen_ids.add(last_id)

	# do stuff for ids without transactions
	allids = set(ids.keys())
	unseen_ids = allids.difference(seen_ids)
	if len(unseen_ids) > 0:
		import pdb;pdb.set_trace()
	for ui in unseen_ids:
		outline = str(ui)
		outline += "0,0,0,0,0,0,0,0\n"
		out_train.write( outline )

generate_features(history_file, transactions_file, out_file)

100000
200000
300000
400000
500000
600000
700000
800000
900000
> <ipython-input-1-2a4573f84b78>(161)generate_features()
    159         if len(unseen_ids) > 0:
    160                 import pdb;pdb.set_trace()
--> 161         for ui in unseen_ids:
    162                 outline = str(ui)
    163                 outline += "0,0,0,0,0,0,0,0\n"



ipdb>  r


--Return--
None
> <ipython-input-1-2a4573f84b78>(164)generate_features()
    162                 outline = str(ui)
    163                 outline += "0,0,0,0,0,0,0,0\n"
--> 164                 out_train.write( outline )
    165 
    166 generate_features(history_file, transactions_file, out_file)



ipdb>  r


TypeError: a bytes-like object is required, not 'str'
> <ipython-input-1-2a4573f84b78>(166)<module>()
    162                 outline = str(ui)
    163                 outline += "0,0,0,0,0,0,0,0\n"
    164                 out_train.write( outline )
    165 
--> 166 generate_features(history_file, transactions_file, out_file)



...

--Return--
<ExecutionRes...> result=None>
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2940)_run_cell()
   2938 
   2939         try:
-> 2940             return runner(coro)
   2941         except BaseException as e:
   2942             info = ExecutionInfo(raw_cell, store_history, silent, shell_futures)



ipdb>  r


> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2897)run_cell()
   2895                 raw_cell, store_history, silent, shell_futures)
   2896         finally:
-> 2897             self.events.trigger('post_execute')
   2898             if not silent:
   2899                 self.events.trigger('post_run_cell', result)



ipdb>  r


--Return--
<ExecutionRes...> result=None>
> c:\programdata\anaconda3\lib\site-packages\ipython\core\interactiveshell.py(2900)run_cell()
   2898             if not silent:
   2899                 self.events.trigger('post_run_cell', result)
-> 2900         return result
   2901 
   2902     def _run_cell(self, raw_cell:str, store_history:bool, silent:bool, shell_futures:bool):

